In [1]:
!rm -rf technosignature-pipeline-v2
!git clone https://github.com/rbaner21/technosignature-pipeline-v2.git
!pip install -q pandas astropy

Cloning into 'technosignature-pipeline-v2'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 89 (delta 31), reused 23 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 32.86 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

# Load planets
master = pd.read_csv("inputs/confirmed_planets.csv")
coords_pl = SkyCoord(master["ra"]*u.deg, master["dec"]*u.deg)

# NEOWISE IR flag (via coords)
df_neo = pd.read_parquet("inputs/neowise_filtered_icrs.parquet")
coords_neo = SkyCoord(df_neo["ra_icrs"]*u.deg, df_neo["dec_icrs"]*u.deg)
idx,d2d,_ = match_coordinates_sky(coords_pl, coords_neo)
master["ir_flag"] = (d2d < 1*u.arcsec)

# BL radio hits
df_bl = pd.read_csv("inputs/bl_filtered.csv")
coords_bl = SkyCoord(df_bl["ra"]*u.deg, df_bl["dec"]*u.deg)
idx,d2d,_ = match_coordinates_sky(coords_pl, coords_bl)
master["radio_flag"] = (d2d < 1*u.arcsec)

# TESS flags by TIC ID
df_tess = pd.read_csv("inputs/tess_flags.csv")
master = master.merge(df_tess[["tic_id","tess_flag"]], on="tic_id", how="left")

# Gaia flags by planet name
df_gaia = pd.read_csv("inputs/gaia_flags.csv")
master = master.merge(
    df_gaia[["planet_id","gaia_flag"]],
    left_on="pl_name", right_on="planet_id",
    how="left"
)

# HARPS spectral spikes (not by coord—just mark if any file exists)
df_spec = pd.read_csv("inputs/spec_flags.csv")
# if ANY HARPS file matched this planet, we could set spec_flag=True.
# Here we just broadcast False (or you can customize)
master["spec_flag"] = False

# Fill missing flags
for col in ["tess_flag","gaia_flag"]:
    master[col] = master[col].fillna(False)

# Save feature matrix
master.to_csv("outputs/feature_matrix.csv", index=False)
print("✅ Feature matrix saved:", len(master), "rows")
